In [1]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow import keras
import numpy as np
from os import listdir
from os.path import isfile
import math

In [2]:
class SequenceGenerator(keras.utils.Sequence):
    """
    A keras Sequence to be used as an image generator for the model.
    """

    def __init__(self, x, y, batchsize):
        self.x, self.y, self.batchsize = x, y, batchsize

    def __len__(self):
        return math.ceil(len(self.x) / self.batchsize)

    def __getitem__(self, idx):
        x_names = self.x[idx * self.batchsize:(idx + 1) * self.batchsize]
        y_names = np.asarray(self.y[idx * self.batchsize:(idx + 1) * self.batchsize])
        
        # open x image names, resize, normalise and make a numpy array
        batch_x = np.asarray([preprocess_input(img_to_array(load_img(file_name, target_size=(299, 299)))) for file_name in x_names])

        return batch_x, y_names

    def num_classes(self):
        ret = []
        for cat in self.y:
            if cat not in ret:
                ret.append(cat)
        return len(ret)

In [3]:
def get_filenames_from_dir(directory, num_categories=1000):
    x = []
    y = []
    category_count = 0
    for category in listdir(directory):
        if isfile(category):
            continue
        for file in listdir("{0}/{1}".format(directory, category)):
            if file[-3:] != "jpg":
                continue
            x.append("{0}/{1}/{2}".format(directory, category, file))
            y.append(category_count)
        category_count += 1
        if category_count == num_categories:
            break

    return x, y

In [4]:
# makes arrays of the images and label names
x_names, y_names = get_filenames_from_dir("database")

# 15% of all the images are set aside as the test set
x_train_val, x_test, y_train_val, y_test = train_test_split(x_names, y_names, test_size=0.15, random_state=42)

# 17% of the non-test images are set aside as the validation set
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.17, random_state=42)

In [5]:
# make generators with batch size 32 for each set
train_gen = SequenceGenerator(x_train, y_train, 32)
val_gen = SequenceGenerator(x_val, y_val, 32)
test_gen = SequenceGenerator(x_test, y_test, 32)

In [69]:
# Inception model
base_model = InceptionV3(weights="imagenet")
for layer in base_model.layers:
    layer.trainable = True

predictions = keras.layers.Dense(train_gen.num_classes(), activation='softmax')(base_model.layers[-2].output)

model = keras.Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [70]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_658 (Conv2D)             (None, 149, 149, 32) 864         input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization_658 (BatchN (None, 149, 149, 32) 96          conv2d_658[0][0]                 
__________________________________________________________________________________________________
activation_658 (Activation)     (None, 149, 149, 32) 0           batch_normalization_658[0][0]    
____________________________________________________________________________________________

In [71]:
model.fit(train_gen, validation_data=val_gen, epochs=5)

Epoch 1/5
1397/1397 [==============================] - 550s 387ms/step - loss: 2.5117 - accuracy: 0.3792 - val_loss: 1.5986 - val_accuracy: 0.5226
Epoch 2/5
1397/1397 [==============================] - 540s 386ms/step - loss: 0.8679 - accuracy: 0.7340 - val_loss: 1.1311 - val_accuracy: 0.6724
Epoch 3/5
1397/1397 [==============================] - 540s 386ms/step - loss: 0.5560 - accuracy: 0.8245 - val_loss: 0.8889 - val_accuracy: 0.7438
Epoch 4/5
1397/1397 [==============================] - 537s 384ms/step - loss: 0.4206 - accuracy: 0.8646 - val_loss: 0.9887 - val_accuracy: 0.7120
Epoch 5/5
1397/1397 [==============================] - 539s 385ms/step - loss: 0.3155 - accuracy: 0.8978 - val_loss: 1.0183 - val_accuracy: 0.7223


In [72]:
model.evaluate(test_gen)

298/298 [==============================] - 55s 184ms/step - loss: 0.9819 - accuracy: 0.7266


[0.981907844543457, 0.7265649437904358]